In [1]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List

# %load_ext nb_black
%load_ext lab_black

## Question 1

In [20]:
toys: List[str] = ["Space", "Zapper", "Big", "Soaker"]

resources: List[str] = ["plastic", "time"]

r: Dict[tuple, float] = {
    ("plastic", "Space"): 2,
    ("plastic", "Zapper"): 1,
    ("plastic", "Big"): 3,
    ("plastic", "Soaker"): 4,
    ("time", "Space"): 3,
    ("time", "Zapper"): 4,
    ("time", "Big"): 5,
    ("time", "Soaker"): 6,
}

availability: Dict[str, int] = {"plastic": 3000, "time": 6000}

profit: Dict[str, int] = {"Space": 16, "Zapper": 15, "Big": 20, "Soaker": 22}

In [21]:
# model
model = gp.Model()
x = model.addVars(toys, obj=profit, ub=1000, lb=100)

model.ModelSense = gp.GRB.MAXIMIZE
resourceCons = model.addConstrs(
    quicksum(r[(i, j)] * x[j] for j in toys) <= availability[i] for i in resources
)
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04


In [22]:
for toy in range(len(toys)):
    print("%s-toys to produce:" % toys[toy], model.x[toy])

Space-toys to produce: 860.0
Zapper-toys to produce: 580.0
Big-toys to produce: 100.0
Soaker-toys to produce: 100.0


#### Using forloops

In [16]:
products: List[str] = ["Space", "Zapper", "Big", "Soaker"]

resources: List[str] = ["plastic", "time"]

resUse: Dict[tuple, float] = {
    ("plastic", "Space"): 2,
    ("plastic", "Zapper"): 1,
    ("plastic", "Big"): 3,
    ("plastic", "Soaker"): 4,
    ("time", "Space"): 3,
    ("time", "Zapper"): 4,
    ("time", "Big"): 5,
    ("time", "Soaker"): 6,
}

resAvail: Dict[str, int] = {"plastic": 3000, "time": 6000}

profit: Dict[str, int] = {"Space": 16, "Zapper": 15, "Big": 20, "Soaker": 22}




In [25]:
model = gp.Model()
pvars = {}


for p in products:
    '''
    For each product set the same constrains of upper lower bound and set the objective 
    Same as: x = model.addVars(toys, obj=profit, ub=1000, lb=100)
    '''
    pvars[p] = model.addVar(lb=100, ub=1000, obj=profit[p], name=p)
model.ModelSense = gp.GRB.MAXIMIZE

for r in resources:
    lhs = 0
    for p in products:
        lhs += resUse[r, p] * pvars[p]
    model.addConstr(lhs <= resAvail[r])


model.optimize()
print("objective value is:", model.ObjVal)


for product in range(len(products)):
    print("%s-toys to produce:" % products[product], model.x[product])

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04
objective value is: 26660.0
Space-toys to produce: 860.0
Zapper-toys to produce: 580.0
Big-toys to produce: 100.0
Soaker-toys to produce: 100.0


## Question 2


Essentially: the process is as follows:
- You need to minimize the ALL costs of you as a provider but also the subcontractors
- a good way to imagine the DECISIONS (what to minimize) is to not think of you as a provider but just: you have 3 different providers 


    

In [18]:
makeCosts = [6, 13, 20]
buyACosts = [12, 15, 21]
buyBCosts = [11, 16, 23]
laborUse = [1, 2, 3]
demands = [10, 80, 70]
aProdLimit = 60
laborAvail = 200
n = 3  # number of vars


model = gp.Model()

M = model.addVars(n, name="make", obj=makeCosts)
A = model.addVars(n, name="BuyA", obj=makeCosts)
B = model.addVars(n, name="BuyB", obj=makeCosts)

# minimize by default --> only use make variables becasue this is the one you are minimizing
model.addConstr(gp.quicksum(laborUse[i] * M[i] for i in range(n)) <= laborAvail)
model.addConstr(gp.quicksum(A[i] for i in range(n)) <= aProdLimit)

# these are multiple constraints now
model.addConstrs(M[i] + A[i] + B[i] == demands[i] for i in range(n))


model.optimize()
print("objective value is:", model.ObjVal)
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 9 columns and 15 nonzeros
Model fingerprint: 0xc5d7eadd
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve time: 0.01s
Presolved: 5 rows, 9 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.500000000e+03
objective value is: 2500.0


## Question 3



## Question 4